In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2023 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

# Custom Liquid - Liquid Extractor Unit Model
Author: Javal Vyas  
Maintainer: Javal Vyas  
Updated: 2023-02-20

This tutorial is a comprehensive step-wise procedure to build a custom unit model from scratch. This tutorial will include creating a property package, a custom unit model and testing them. For this tutorial we shall create a custom unit model for Liquid - Liquid Extraction.  

The Liquid - Liquid Extractor model contains two immiscible fluids forming the two phases. One of the phase, say phase_1 has a high concentration of solutes which is to be separated. A mass transfer happens between the two phases and the solute is transfered from phase_1 to phase_2. This mass_transfer is governed by a parameter called distribution co-efficient. 

After gaining the working principles of the Liquid - Liquid Extractor, we shall proceed to create a custom unit model, we would require a property package for each phase, a custom unit model class and then write the tests for the model and property packages. 

Before commencing the development of the model, we need to state some assumptions which the following unit model will be using. 
- Steady-state only
- Liquid phase property package has a single phase named Liq
- Aquoeus phase property package has a single phase named Aq
- Liquid and Aqueous phase properties need not have the same component list. 

Thus as per the assumptions, we will be creating one property package for the aqueous phase (Aq), and the other for the liquid phase (Liq). 
<!-- This Liquid - Liquid Extractor model includes aqueous and organic phase outlets. This is inspired from the anaerobic_digestor in [watertap](https://github.com/watertap-org/watertap/blob/43eca896726b0f2983e2761c1141ae5504d3626f/watertap/unit_models/anaerobic_digester.py#L4) repository. This tutorial demonstrates how to handle two phase systems with two inlet ports and two outlet ports. In this tutorial we shall solve the separation of salts from the aqueous phase to the liquid phase. 

The unit model uses two property packages, one for the liquid phase and another for the aqueous phase. It also has two inlet ports and two outlet ports, each for a phase respectively. The solutes get extracted from aqueous phase as per the distribution co-efficient.   -->


# 1. Creating Liquid Property Package

Creating a property package is a 4 step process
- Import necessary libraries 
- Creating Physical Parameter Data Block
- Define State Block
- Define State Block Data

# 1.1 Importing necessary packages 
Let us begin with the importing the necessary libraries where we will be using functionalities from idaes and pyomo. 

In [2]:
# Import Python libraries
import logging

import idaes.logger as idaeslog
from idaes.core.util.initialization import fix_state_vars, revert_state_vars

# Import Pyomo libraries
from pyomo.environ import Param, Var, NonNegativeReals, units, Expression, PositiveReals

# Import IDAES cores
from idaes.core import (
    declare_process_block_class,
    MaterialFlowBasis,
    PhysicalParameterBlock,
    StateBlockData,
    StateBlock,
    MaterialBalanceType,
    EnergyBalanceType,
    Solute,
    Solvent,
    LiquidPhase,
)
from idaes.core.util.model_statistics import degrees_of_freedom

# 1.2 Physical Parameter Data Block

A Physical Parameter Block serves as the central point of reference for all aspects of the property package and needs to define several things about the package. These are summarized below:

- Units of measurement
- What properties are supported and how they are implemented
- What components and phases are included in the packages
- All the global parameters necessary for calculating properties
- A reference to the associated State Block class, so that construction of the State Block components can be automated from the Physical Parameter Block

To construct this block, we begin by declaring a process block class using a Python decorator. One can learn more about `declare_process_block_class` [here](https://github.com/IDAES/idaes-pse/blob/eea1209077b75f7d940d8958362e69d4650c079d/idaes/core/base/process_block.py#L173). After constructing the process block, we define a build function which contains all the components that the property package would have. `super` function here is used to give access to methods and properties of a parent or sibling class and since this is used on the class `PhysicalParameterData` class, build has access to all the parent and sibling class methods. 

The Physical Parameter Block then refers to the `state block` in this case `LiqPhaseStateBlock` (which will be declared later), so that we can build a state block instance by only knowing the `PhysicalParameterBlock` we wish to use. Then we move on to list the number of phases in this property package. Then we assign the variable to the phase which follows a naming convention. Like here since the solvent is in the liquid phase, we will assign the Phase as LiquidPhase and the variable will be named Liq as per the naming convention. The details of naming conventions can be found [here](https://github.com/IDAES/idaes-pse/blob/main/docs/explanations/conventions.rst). We will be following the same convention throughout the example. 
 
After defining the list of the phases, we move on to list the components and their type in the phase. It can be a solute or a solvent in the liquid phase. Thus, we define the component and assign it to either being a solute or a solvent. In this case, the salts are the solutes and Ethylene dibromide is the solvent. Next, we define the physical properties involved in the package, like the heat capacity and density of the solvent, the reference temperature, and the distribution factor that would govern the mass transfer from one phase into another. 

The final step in creating the Physical Parameter Block is to declare a `classmethod` named `define_metadata`, which takes two arguments: a class (cls) and an instance of that class (obj). This method in turn needs to call two predefined methods (inherited from underlying base classes):

- `obj.add_properties()` is used to set the metadata regarding the supported properties, and here we define flow volume, pressure, temperature, and mass fraction.
- `obj.add_default_units()` sets the default units metadata for the property package, and here we define units to be used with this property package as default.

In [3]:
@declare_process_block_class("LiqPhase")
class PhysicalParameterData(PhysicalParameterBlock):
    """
    Property Parameter Block Class

    Contains parameters and indexing sets associated with properties for
    liquid Phase

    """

    def build(self):
        """
        Callable method for Block construction.
        """
        super(PhysicalParameterData, self).build()

        self._state_block_class = LiqPhaseStateBlock

        # List of valid phases in property package
        self.Liq = LiquidPhase()

        # Component list - a list of component identifiers
        self.NaCl = Solute()
        self.KNO3 = Solute()
        self.CaSO4 = Solute()
        self.C2H4Br2 = (
            Solvent()
        )  # Solvent used here is ethylene dibromide (Organic Polar)

        # Heat capacity of solvent
        self.cp_mass = Param(
            mutable=True,
            initialize=717.01,
            doc="Specific heat capacity of solvent",
            units=units.J / units.kg / units.K,
        )

        # Density of solvent
        self.dens_mass = Param(
            mutable=True,
            initialize=2170,
            doc="Density of ethylene dibromide",
            units=units.kg / units.m**3,
        )

        # Reference Temperature
        self.temperature_ref = Param(
            within=PositiveReals,
            mutable=True,
            default=298.15,
            doc="Reference temperature",
            units=units.K,
        )

        # Distribution Factor
        salts_d = {"NaCl": 2.15, "KNO3": 3, "CaSO4": 1.5}
        self.diffusion_factor = Param(
            salts_d.keys(), initialize=salts_d, within=PositiveReals
        )

    @classmethod
    def define_metadata(cls, obj):
        obj.add_properties(
            {
                "flow_vol": {"method": None, "units": "kmol/s"},
                "pressure": {"method": None, "units": "MPa"},
                "temperature": {"method": None, "units": "K"},
                "conc_mass_comp": {"method": None},
            }
        )

        obj.add_default_units(
            {
                "time": units.s,
                "length": units.m,
                "mass": units.kg,
                "amount": units.mol,
                "temperature": units.K,
            }
        )

# 1.3 State Block

After the `Physical Parameter Block` class has been created, the next step is to write the code necessary to create the State Blocks that will be used throughout the flowsheet. `StateBlock` contains all the information necessary to define the state of the system. This includes the state variables and constraints on those variables which are used to describe a state property like the enthalpy, material balance, etc. 

Creating a State Block requires us to write two classes. The reason we write two classes is because of the inherent nature of how `declare_process_block_data` works. `declare_process_block_data` facilitates creating an `IndexedComponent` object which can handle multiple `ComponentData` objects which represent the component at each point in the indexing set. This makes it easier to build an instance of the model at each indexed point. However, State Blocks are slightly different, as they are always indexed (at least by time). Due to this, we often want to perform actions on all the elements of the indexed `StateBlock` all at once (rather than element by element), such as initialization. 

The class `_StateBlock` is defined without the `declare_process_block_data` decorator and thus works as a traditional class and this facilitates performing a method on the class as a whole rather than individual elements of the indexed property blocks. In this class we define the initialization routine and release_state function. Initialization is of extreme importance when it comes to non-linear programs like the one, we have on our hands. The initialization function takes initial guesses for the state variables chosen as an argument `state_args`. This has the following inputs for this unit model, initial guesses for the following state variables, flow_mol_comp, temperature, pressure. This also takes a boolean argument for if the state variable is fixed or not in `state_vars_fixed`. If `state_vars_fixed` is `True` then the initialization routine will not deal with fixing and unfixing variables. `hold_state` is a boolean argument that indicates if the state variables should be unfixed during initialization. If this argument is `False` then no state variables are fixed, else the state variables are fixed, and it returns a dictionary with flags for the state variables indicating which are fixed during the initialization. `outlvl` is an output level of logging. The other 2 arguments are `solver` and `optarg` which are the solver and solver options respectively. 

When we look further into the initialization routine, we see that we write a logger. This helps debug the property package. The next step is to check for the state variables if they are fixed or not. If the state variables are not fixed then we fix them using `fix_state_vars` function and if they are fixed we check the degrees of freedom (dof) for the initialization routine. We need to have dof be 0 for the initialization routine and if the dof is not 0, we raise exception. The next step in the initialization routine will be to take into account the `hold_state` argument. This argument will only make sense if the `fix_state_vars` argument is `False`. We check for the same and return the flags for the states if `hold_state` is `True`, else the state is released using `release_state` function. This concludes the initialization routine, and we log that the initialization routine is complete. 


`release_states` function is to used to free the state variable from the current state. It takes flags as the argument which determine if the state is to held or can be released from initialize function. This function uses `revert_state_vars` function which reverts the fixed state of the state variables within an IDAES StateBlock based on a set of flags of the previous state. After releasing the state variables, completion of state release is logged. 

The above two functions comprise of the `_StateBlock`, next we shall see the construction of the `LiqPhaseStateBlockData` class. 

In [4]:
class _StateBlock(StateBlock):
    """
    This Class contains methods which should be applied to Property Blocks as a
    whole, rather than individual elements of indexed Property Blocks.
    """

    def initialize(
        self,
        state_args=None,
        state_vars_fixed=False,
        hold_state=False,
        outlvl=idaeslog.NOTSET,
        solver=None,
        optarg=None,
    ):
        """
        Initialization routine for property package.

        Keyword Arguments:
            state_args : Dictionary with initial guesses for the state vars
                         chosen. Note that if this method is triggered
                         through the control volume, and if initial guesses
                         were not provided at the unit model level, the
                         control volume passes the inlet values as initial
                         guess.The keys for the state_args dictionary are:
            flow_mol_comp : value at which to initialize component flows (default=None)
            pressure : value at which to initialize pressure (default=None)
            temperature : value at which to initialize temperature (default=None)
            outlvl : sets output level of initialization routine
            state_vars_fixed: Flag to denote if state vars have already been fixed.
                              True - states have already been fixed and
                              initialization does not need to worry
                              about fixing and unfixing variables.
                              False - states have not been fixed. The state
                              block will deal with fixing/unfixing.
            optarg : solver options dictionary object (default=None, use
                     default solver options)
            solver : str indicating which solver to use during
                     initialization (default = None, use default solver)
            hold_state : flag indicating whether the initialization routine
                         should unfix any state variables fixed during
                         initialization (default=False).
                         True - states variables are not unfixed, and
                         a dict of returned containing flags for
                         which states were fixed during initialization.
                         False - state variables are unfixed after
                         initialization by calling the
                         release_state method

        Returns:
            If hold_states is True, returns a dict containing flags for
            which states were fixed during initialization.
        """
        init_log = idaeslog.getInitLogger(self.name, outlvl, tag="properties")

        if state_vars_fixed is False:
            # Fix state variables if not already fixed
            flags = fix_state_vars(self, state_args)

        else:
            # Check when the state vars are fixed already result in dof 0
            for k in self.keys():
                if degrees_of_freedom(self[k]) != 0:
                    raise Exception(
                        "State vars fixed but degrees of freedom "
                        "for state block is not zero during "
                        "initialization."
                    )

        if state_vars_fixed is False:
            if hold_state is True:
                return flags
            else:
                self.release_state(flags)

        init_log.info("Initialization Complete.")

    def release_state(self, flags, outlvl=idaeslog.NOTSET):
        """
        Method to release state variables fixed during initialization.

        Keyword Arguments:
            flags : dict containing information of which state variables
                    were fixed during initialization, and should now be
                    unfixed. This dict is returned by initialize if
                    hold_state=True.
            outlvl : sets output level of logging
        """
        init_log = idaeslog.getInitLogger(self.name, outlvl, tag="properties")

        if flags is None:
            return
        # Unfix state variables
        revert_state_vars(self, flags)
        init_log.info("State Released.")

The class `LiqPhaseStateBlockData` is designated with the `declare_process_block_class` decorator, named `LiqPhaseStateBlock`, and inherits the block class from `_StateBlock`. This inheritance allows `LiqPhaseStateBlockData` to leverage functions from `_StateBlock`. Following the class definition, a build function similar to the one used in the `PhysicalParameterData` block is employed. The super function is utilized to enable the utilization of functions from the parent or sibling class.

The subsequent objective is to delineate the state variables, accomplished through the `_make_state_vars` method. This method encompasses all the essential state variables and associated data. For this particular property package, the required state variables are:

- `flow_vol` - volumetric flow rate
- `conc_mass_comp` - mass fractions
- `pressure` - state pressure
- `temperature` - state temperature

Additionally, a state parameter, the `diffusion_factor`, is introduced. This factor plays a crucial role in governing mass transfer between phases, necessitating its definition within the state block.

After establishing the state variables, the subsequent step involves setting up state properties as constraints. This includes specifying the relationships and limitations that dictate the system's behavior. The following properties need to be articulated:

- `material flow:` quantifies the amount of material flow.
- `enthalpy flow:` specifies the amount of enthalpy flow.
- `mass composition:` defines mass fractions.
- `flowrates:` details volumetric flow rates.
- `material balance:` ensures the total material balance is maintained.
- `energy balance:` ensures the total energy balance is preserved.
- `state variables:` involves defining state variables with units, akin to the `define_metadata` function in the `PhysicalParameterData` block.
- `state variable basis:` establishes the basis on which state variables are measured, whether in mass or molar terms.

These definitions mark the conclusion of the state block construction and thus the property package. For additional details on creating a property package, please refer to [this resource](../../properties/custom/custom_physical_property_packages_doc.md).

In [5]:
@declare_process_block_class("LiqPhaseStateBlock", block_class=_StateBlock)
class LiqPhaseStateBlockData(StateBlockData):
    """
    An example property package for liquid phase for liquid liquid extraction
    """

    def build(self):
        """
        Callable method for Block construction
        """
        super(LiqPhaseStateBlockData, self).build()
        self._make_state_vars()

    def _make_state_vars(self):
        salts_d = {"NaCl": 2.15, "KNO3": 3, "CaSO4": 1.5}
        self.flow_vol = Var(
            initialize=1,
            domain=NonNegativeReals,
            doc="Total volumetric flowrate",
            units=units.ml / units.min,
        )
        self.conc_mass_comp = Var(
            salts_d.keys(),
            domain=NonNegativeReals,
            initialize=1,
            doc="Component mass concentrations",
            units=units.g / units.kg,
        )
        self.pressure = Var(
            domain=NonNegativeReals,
            initialize=1,
            bounds=(1, 5),
            units=units.atm,
            doc="State pressure [atm]",
        )
        self.temperature = Var(
            domain=NonNegativeReals,
            initialize=300,
            bounds=(273, 373),
            units=units.K,
            doc="State temperature [K]",
        )
        self.diffusion_factor = Param(
            salts_d.keys(),
            initialize=salts_d,
            doc="Diffusion Factor of salts",
            within=PositiveReals,
        )

        def material_flow_expression(self, j):
            if j == "solvent":
                return self.flow_vol * self.params.dens_mass
            else:
                return self.conc_mass_comp[j]

        self.material_flow_expression = Expression(
            self.component_list,
            rule=material_flow_expression,
            doc="Material flow terms",
        )

        def enthalpy_flow_expression(self):
            return (
                self.flow_vol
                * self.params.dens_mass
                * self.params.cp_mass
                * (self.temperature - self.params.temperature_ref)
            )

        self.enthalpy_flow_expression = Expression(
            rule=enthalpy_flow_expression, doc="Enthalpy flow term"
        )

    def get_mass_comp(self, j):
        return self.conc_mass_comp[j]

    def get_flow_rate(self):
        return self.flow_vol

    def get_material_flow_terms(self, p, j):
        return self.material_flow_expression[j]

    def get_enthalpy_flow_terms(self, p):
        return self.enthalpy_flow_expression

    def default_material_balance_type(self):
        return MaterialBalanceType.componentTotal

    def default_energy_balance_type(self):
        return EnergyBalanceType.enthalpyTotal

    def define_state_vars(self):
        return {
            "flow_vol": self.flow_vol,
            "conc_mass_comp": self.conc_mass_comp,
            "temperature": self.temperature,
            "pressure": self.pressure,
        }

    def get_material_flow_basis(self):
        return MaterialFlowBasis.mass

# 2. Creating Aqueous Property Package

The structure of Aqueous Property Package mirrors that of the Liquid Property Package we previously developed. We'll commence with an overview, importing the required libraries, followed by the creation of the physical property block and two state blocks. The distinctions in this package lie in the physical parameter values, and notably, the absence of the diffusion factor term, differentiating it from the prior package. The following code snippet should provide clarity on these distinctions.

In [6]:
# Changes the divide behavior to not do integer division
from __future__ import division

# Import Python libraries
import logging

import idaes.logger as idaeslog
from idaes.core.util.initialization import fix_state_vars, revert_state_vars

# Import Pyomo libraries
from pyomo.environ import Param, Var, NonNegativeReals, units, Expression, PositiveReals

# Import IDAES cores
from idaes.core import (
    declare_process_block_class,
    MaterialFlowBasis,
    PhysicalParameterBlock,
    StateBlockData,
    StateBlock,
    MaterialBalanceType,
    EnergyBalanceType,
    Solute,
    Solvent,
    LiquidPhase,
)
from idaes.core.util.model_statistics import degrees_of_freedom

# Some more information about this module
__author__ = "Javal Vyas"


# Set up logger
_log = logging.getLogger(__name__)


@declare_process_block_class("AqPhase")
class PhysicalParameterData(PhysicalParameterBlock):
    """
    Property Parameter Block Class

    Contains parameters and indexing sets associated with properties for
    liquid Phase

    """

    def build(self):
        """
        Callable method for Block construction.
        """
        super(PhysicalParameterData, self).build()

        self._state_block_class = AqPhaseStateBlock

        # List of valid phases in property package
        self.Aq = LiquidPhase()

        # Component list - a list of component identifiers
        self.NaCl = Solute()
        self.KNO3 = Solute()
        self.CaSO4 = Solute()
        self.H2O = Solvent()

        # Heat capacity of Water
        self.cp_mass = Param(
            mutable=True,
            initialize=4182,
            doc="Specific heat capacity of solvent",
            units=units.J / units.kg / units.K,
        )

        # Density of water
        self.dens_mass = Param(
            mutable=True,
            initialize=997,
            doc="Density of ethylene dibromide",
            units=units.kg / units.m**3,
        )

        # Reference temperature
        self.temperature_ref = Param(
            within=PositiveReals,
            mutable=True,
            default=298.15,
            doc="Reference temperature",
            units=units.K,
        )

    @classmethod
    def define_metadata(cls, obj):
        obj.add_properties(
            {
                "flow_mol": {"method": None, "units": "kmol/s"},
                "pressure": {"method": None, "units": "MPa"},
                "temperature": {"method": None, "units": "K"},
            }
        )

        obj.add_default_units(
            {
                "time": units.s,
                "length": units.m,
                "mass": units.kg,
                "amount": units.mol,
                "temperature": units.K,
            }
        )


class _StateBlock(StateBlock):
    """
    This Class contains methods which should be applied to Property Blocks as a
    whole, rather than individual elements of indexed Property Blocks.
    """

    def initialize(
        self,
        state_args=None,
        state_vars_fixed=False,
        hold_state=False,
        outlvl=idaeslog.NOTSET,
        solver=None,
        optarg=None,
    ):
        """
        Initialization routine for property package.

        Keyword Arguments:
            state_args : Dictionary with initial guesses for the state vars
                         chosen. Note that if this method is triggered
                         through the control volume, and if initial guesses
                         were not provided at the unit model level, the
                         control volume passes the inlet values as initial
                         guess.The keys for the state_args dictionary are:
            flow_mol_comp : value at which to initialize component flows (default=None)
            pressure : value at which to initialize pressure (default=None)
            temperature : value at which to initialize temperature (default=None)
            outlvl : sets output level of initialization routine
            state_vars_fixed: Flag to denote if state vars have already been fixed.
                              True - states have already been fixed and
                              initialization does not need to worry
                              about fixing and unfixing variables.
                              False - states have not been fixed. The state
                              block will deal with fixing/unfixing.
            optarg : solver options dictionary object (default=None, use
                     default solver options)
            solver : str indicating which solver to use during
                     initialization (default = None, use default solver)
            hold_state : flag indicating whether the initialization routine
                         should unfix any state variables fixed during
                         initialization (default=False).
                         True - states variables are not unfixed, and
                         a dict of returned containing flags for
                         which states were fixed during initialization.
                         False - state variables are unfixed after
                         initialization by calling the
                         release_state method

        Returns:
            If hold_states is True, returns a dict containing flags for
            which states were fixed during initialization.
        """
        init_log = idaeslog.getInitLogger(self.name, outlvl, tag="properties")

        if state_vars_fixed is False:
            # Fix state variables if not already fixed
            flags = fix_state_vars(self, state_args)

        else:
            # Check when the state vars are fixed already result in dof 0
            for k in self.keys():
                if degrees_of_freedom(self[k]) != 0:
                    raise Exception(
                        "State vars fixed but degrees of freedom "
                        "for state block is not zero during "
                        "initialization."
                    )

        if state_vars_fixed is False:
            if hold_state is True:
                return flags
            else:
                self.release_state(flags)

        init_log.info("Initialization Complete.")

    def release_state(self, flags, outlvl=idaeslog.NOTSET):
        """
        Method to release state variables fixed during initialization.

        Keyword Arguments:
            flags : dict containing information of which state variables
                    were fixed during initialization, and should now be
                    unfixed. This dict is returned by initialize if
                    hold_state=True.
            outlvl : sets output level of logging
        """
        init_log = idaeslog.getInitLogger(self.name, outlvl, tag="properties")

        if flags is None:
            return
        # Unfix state variables
        revert_state_vars(self, flags)
        init_log.info("State Released.")


@declare_process_block_class("AqPhaseStateBlock", block_class=_StateBlock)
class AqPhaseStateBlockData(StateBlockData):
    """
    An example property package for aqueous phase for liquid liquid extraction
    """

    def build(self):
        """
        Callable method for Block construction
        """
        super(AqPhaseStateBlockData, self).build()
        self._make_state_vars()

    def _make_state_vars(self):
        self.flow_vol = Var(
            initialize=1,
            domain=NonNegativeReals,
            doc="Total volumetric flowrate",
            units=units.ml / units.min,
        )

        salts_conc = {"NaCl": 0.15, "KNO3": 0.2, "CaSO4": 0.1}
        self.conc_mass_comp = Var(
            salts_conc.keys(),
            domain=NonNegativeReals,
            initialize=1,
            doc="Component mass concentrations",
            units=units.g / units.kg,
        )
        self.pressure = Var(
            domain=NonNegativeReals,
            initialize=1,
            bounds=(1, 5),
            units=units.atm,
            doc="State pressure [atm]",
        )

        self.temperature = Var(
            domain=NonNegativeReals,
            initialize=300,
            bounds=(273, 373),
            units=units.K,
            doc="State temperature [K]",
        )

        def material_flow_expression(self, j):
            if j == "H2O":
                return self.flow_vol * self.params.dens_mass
            else:
                return self.flow_vol * self.conc_mass_comp[j]

        self.material_flow_expression = Expression(
            self.component_list,
            rule=material_flow_expression,
            doc="Material flow terms",
        )

        def enthalpy_flow_expression(self):
            return (
                self.flow_vol
                * self.params.dens_mass
                * self.params.cp_mass
                * (self.temperature - self.params.temperature_ref)
            )

        self.enthalpy_flow_expression = Expression(
            rule=enthalpy_flow_expression, doc="Enthalpy flow term"
        )

    def get_mass_comp(self, j):
        return self.conc_mass_comp[j]

    def get_flow_rate(self):
        return self.flow_vol

    def get_material_flow_terms(self, p, j):
        return self.material_flow_expression[j]

    def get_enthalpy_flow_terms(self, p):
        return self.enthalpy_flow_expression

    def default_material_balance_type(self):
        return MaterialBalanceType.componentTotal

    def default_energy_balance_type(self):
        return EnergyBalanceType.enthalpyTotal

    def define_state_vars(self):
        return {
            "flow_vol": self.flow_vol,
            "conc_mass_comp": self.conc_mass_comp,
            "temperature": self.temperature,
            "pressure": self.pressure,
        }

    def get_material_flow_basis(self):
        return MaterialFlowBasis.mass

# 3. Liquid Liquid Extractor Unit Model

Following the creation of property packages, our next step is to develop a unit model that facilitates the mass transfer of solutes between phases. This involves importing necessary libraries, building the unit model, defining auxiliary functions, and establishing the initialization routine for the unit model.

## 3.1 Importing necessary libraries

Let's commence by importing the essential libraries from pyomo and idaes.

In [7]:
# Import Pyomo libraries
from pyomo.common.config import ConfigBlock, ConfigValue, In, Bool
from pyomo.environ import (
    Reference,
    Var,
    value,
    Constraint,
    units as pyunits,
    check_optimal_termination,
    Suffix,
)

# Import IDAES cores
from idaes.core import (
    ControlVolume0DBlock,
    declare_process_block_class,
    MaterialBalanceType,
    EnergyBalanceType,
    MaterialFlowBasis,
    MomentumBalanceType,
    UnitModelBlockData,
    useDefault,
)
from idaes.core.util.config import (
    is_physical_parameter_block,
    is_reaction_parameter_block,
)

import idaes.logger as idaeslog
from idaes.core.util import scaling as iscale
from idaes.core.solvers import get_solver
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.exceptions import ConfigurationError, InitializationError

## 3.2 Creating the unit model

Creating a unit model starts by creating a class called `LiqExtractionData` and use the `declare_process_block_class` decorator. The `LiqExtractionData` inherts the properties of `UnitModelBlockData` class, which allows us to create a control volume which is necessary for the unit model. After declaration of the class we proceed to define the config arguments for the control volume. The config arguments includes the following properties:

1. `material_balance_type` - Indicates what type of mass balance should be constructed
2. `energy_balance_type` - Indicates what type of energy balance should be constructed
3. `momentum_balance_type` - Indicates what type of momentum balance should be constructed
4. `has_heat_transfer` - Indicates whether terms for heat transfer should be constructed
5. `has_pressure_change` - Indicates whether terms for pressure change should be
constructed
6. `has_equilibrium_reactions` - Indicates whether terms for equilibrium controlled reactions
should be constructed
7. `has_phase_equilibrium` - Indicates whether terms for phase equilibrium should be
constructed
8. `has_heat_of_reaction` - Indicates whether terms for heat of reaction terms should be
constructed
9. `Liquid Property` - Property parameter object used to define property calculations
for the liquid phase
10. `Liquid Property Arguments` - Arguments to use for constructing liquid phase properties
11. `Aqueous Property` - Property parameter object used to define property calculations
for the aqueous phase
12. `Aqueous Property Arguments` - Arguments to use for constructing aqueous phase properties
13. `Reaction Package` - Reaction parameter object used to define reaction calculations
14. `Reaction Package Arguments` - Arguments to use for constructing reaction packages


In [8]:
@declare_process_block_class("LiqExtraction")
class LiqExtractionData(UnitModelBlockData):
    """
    LiqExtraction Unit Model Class
    """

    CONFIG = UnitModelBlockData.CONFIG()

    CONFIG.declare(
        "material_balance_type",
        ConfigValue(
            default=MaterialBalanceType.useDefault,
            domain=In(MaterialBalanceType),
            description="Material balance construction flag",
            doc="""Indicates what type of mass balance should be constructed,
**default** - MaterialBalanceType.useDefault.
**Valid values:** {
**MaterialBalanceType.useDefault - refer to property package for default
balance type
**MaterialBalanceType.none** - exclude material balances,
**MaterialBalanceType.componentPhase** - use phase component balances,
**MaterialBalanceType.componentTotal** - use total component balances,
**MaterialBalanceType.elementTotal** - use total element balances,
**MaterialBalanceType.total** - use total material balance.}""",
        ),
    )
    CONFIG.declare(
        "energy_balance_type",
        ConfigValue(
            default=EnergyBalanceType.useDefault,
            domain=In(EnergyBalanceType),
            description="Energy balance construction flag",
            doc="""Indicates what type of energy balance should be constructed,
**default** - EnergyBalanceType.useDefault.
**Valid values:** {
**EnergyBalanceType.useDefault - refer to property package for default
balance type
**EnergyBalanceType.none** - exclude energy balances,
**EnergyBalanceType.enthalpyTotal** - single enthalpy balance for material,
**EnergyBalanceType.enthalpyPhase** - enthalpy balances for each phase,
**EnergyBalanceType.energyTotal** - single energy balance for material,
**EnergyBalanceType.energyPhase** - energy balances for each phase.}""",
        ),
    )
    CONFIG.declare(
        "momentum_balance_type",
        ConfigValue(
            default=MomentumBalanceType.pressureTotal,
            domain=In(MomentumBalanceType),
            description="Momentum balance construction flag",
            doc="""Indicates what type of momentum balance should be constructed,
**default** - MomentumBalanceType.pressureTotal.
**Valid values:** {
**MomentumBalanceType.none** - exclude momentum balances,
**MomentumBalanceType.pressureTotal** - single pressure balance for material,
**MomentumBalanceType.pressurePhase** - pressure balances for each phase,
**MomentumBalanceType.momentumTotal** - single momentum balance for material,
**MomentumBalanceType.momentumPhase** - momentum balances for each phase.}""",
        ),
    )
    CONFIG.declare(
        "has_heat_transfer",
        ConfigValue(
            default=False,
            domain=Bool,
            description="Heat transfer term construction flag",
            doc="""Indicates whether terms for heat transfer should be constructed,
**default** - False.
**Valid values:** {
**True** - include heat transfer terms,
**False** - exclude heat transfer terms.}""",
        ),
    )
    CONFIG.declare(
        "has_pressure_change",
        ConfigValue(
            default=False,
            domain=Bool,
            description="Pressure change term construction flag",
            doc="""Indicates whether terms for pressure change should be
constructed,
**default** - False.
**Valid values:** {
**True** - include pressure change terms,
**False** - exclude pressure change terms.}""",
        ),
    )
    CONFIG.declare(
        "has_equilibrium_reactions",
        ConfigValue(
            default=False,
            domain=Bool,
            description="Equilibrium reaction construction flag",
            doc="""Indicates whether terms for equilibrium controlled reactions
should be constructed,
**default** - True.
**Valid values:** {
**True** - include equilibrium reaction terms,
**False** - exclude equilibrium reaction terms.}""",
        ),
    )
    CONFIG.declare(
        "has_phase_equilibrium",
        ConfigValue(
            default=False,
            domain=Bool,
            description="Phase equilibrium construction flag",
            doc="""Indicates whether terms for phase equilibrium should be
constructed,
**default** = False.
**Valid values:** {
**True** - include phase equilibrium terms
**False** - exclude phase equilibrium terms.}""",
        ),
    )
    CONFIG.declare(
        "has_heat_of_reaction",
        ConfigValue(
            default=False,
            domain=Bool,
            description="Heat of reaction term construction flag",
            doc="""Indicates whether terms for heat of reaction terms should be
constructed,
**default** - False.
**Valid values:** {
**True** - include heat of reaction terms,
**False** - exclude heat of reaction terms.}""",
        ),
    )
    CONFIG.declare(
        "liquid_property_package",
        ConfigValue(
            default=useDefault,
            domain=is_physical_parameter_block,
            description="Property package to use for liquid phase",
            doc="""Property parameter object used to define property calculations
for the liquid phase,
**default** - useDefault.
**Valid values:** {
**useDefault** - use default package from parent model or flowsheet,
**PropertyParameterObject** - a PropertyParameterBlock object.}""",
        ),
    )
    CONFIG.declare(
        "liquid_property_package_args",
        ConfigBlock(
            implicit=True,
            description="Arguments to use for constructing liquid phase properties",
            doc="""A ConfigBlock with arguments to be passed to liquid phase
property block(s) and used when constructing these,
**default** - None.
**Valid values:** {
see property package for documentation.}""",
        ),
    )
    CONFIG.declare(
        "aqueous_property_package",
        ConfigValue(
            default=useDefault,
            domain=is_physical_parameter_block,
            description="Property package to use for aqueous phase",
            doc="""Property parameter object used to define property calculations
for the aqueous phase,
**default** - useDefault.
**Valid values:** {
**useDefault** - use default package from parent model or flowsheet,
**PropertyParameterObject** - a PropertyParameterBlock object.}""",
        ),
    )
    CONFIG.declare(
        "aqueous_property_package_args",
        ConfigBlock(
            implicit=True,
            description="Arguments to use for constructing aqueous phase properties",
            doc="""A ConfigBlock with arguments to be passed to aqueous phase
property block(s) and used when constructing these,
**default** - None.
**Valid values:** {
see property package for documentation.}""",
        ),
    )
    CONFIG.declare(
        "reaction_package",
        ConfigValue(
            default=None,
            domain=is_reaction_parameter_block,
            description="Reaction package to use for control volume",
            doc="""Reaction parameter object used to define reaction calculations,
**default** - None.
**Valid values:** {
**None** - no reaction package,
**ReactionParameterBlock** - a ReactionParameterBlock object.}""",
        ),
    )
    CONFIG.declare(
        "reaction_package_args",
        ConfigBlock(
            implicit=True,
            description="Arguments to use for constructing reaction packages",
            doc="""A ConfigBlock with arguments to be passed to a reaction block(s)
and used when constructing these,
**default** - None.
**Valid values:** {
see reaction package for documentation.}""",
        ),
    )

### Building the model

After constructing the `LiqExtractionData` block and defining the config arguments for the control block, the next step is to write a build function that incorporates control volume and establishes constraints on the control volume to achieve the desired mass transfer. The control volume serves as a pivotal component in the unit model construction, representing the volume in which the process unfolds.

IDAES provides flexibility in choosing control volumes based on geometry, with options including 0D or 1D. In this instance, we opt for a 0D control volume, the most commonly used control volume. This choice is suitable for systems where there is a well-mixed volume of fluid or where spatial variations are deemed negligible.

The control volume encompasses parameters from (1-8), and its equations are configured to satisfy the specified config arguments. For a more in-depth understanding, users are encouraged to refer to [this resource](https://github.com/IDAES/idaes-pse/blob/2f34dd3abc1bce5ba17c80939a01f9034e4fbeef/docs/reference_guides/core/control_volume_0d.rst). 

The `build` function is initiated using the `super` function to gain access to methods and properties of a parent or sibling class, in this case, the `LiqExtractionData` class. Following the `super` function, checks are performed on the property packages to ensure the appropriate names for the solvents, such as 'Aq' for the aqueous phase and 'Liq' for the liquid phase. An error is raised if these conditions are not met. Subsequently, a check is performed to ensure there is at least one common component between the two property packages that can be transferred from one phase to another.

After these checks are completed without any exceptions raised, it is ensured that the property packages have the desired components with appropriate names. The next step is to create a control volume and assign it to a property package. Here, we initiate with the liquid phase and attach a 0D control volume to it. The control volume takes arguments about the dynamics of the block, the hold-up in the block, and the property package, along with property package arguments. 

The subsequent steps involve adding inlet and outlet state blocks to the control volume using the `add_state_blocks` function. This function takes arguments about the flow direction (defaulted to forward) and a flag for `has_phase_equilibrium`, which is read from the config. The control volume is now equipped with the inlet and outlet state blocks and has access to the liquid property package

Next, material balance equations are added to the control volume using the `add_material_balance` function, taking into account the type of material balance, `has_rate_reactions`, `has_phase_equilibrium`, and the presence of `has_mass_transfer`. To understand this arguments further let us have a look at the material balance equation and how it is implemented in control volume. 

$\frac{\partial M_{t, p, j}}{\partial t} = F_{in, t, p, j} - F_{out, t, p, j} + N_{kinetic, t, p, j} + N_{equilibrium, t, p, j} + N_{pe, t, p, j} + N_{transfer, t, p, j} + N_{custom, t, p, j}$

Here we shall see that $N_{transfer, t, p, j}$ is the term in the equation which is reponsible for the mass transfer and the `mass_transfer_term` should only be equal to the amount being transferred and not include a material balance on our own. For a detailed description of the terms one should refer to the following [resource](https://github.com/IDAES/idaes-pse/blob/2f34dd3abc1bce5ba17c80939a01f9034e4fbeef/docs/reference_guides/core/control_volume_0d.rst)

Similarly, `add_energy_balance` and `add_momentum_balance` functions are added to the control volume to create respective equations. This concludes the creation of liquid phase control volume. Similar procedure is done for the aqueous phase control volume with aqueous property package. 

Now, the unit model has two control volumes with appropriate configurations and material, momentum and energy balances. The next step is to check the basis of the two property packages. They should both have the same flow basis, and an error is raised if this is not the case.

Following this, the `add_inlet_ports` and `add_outlet_ports` functions are used to create inlet and outlet ports. These ports are named and assigned to each control volume, resulting in labeled inlet and outlet ports for each control volume.

The subsequent steps involve writing unit-level constraints. A check if the basis is either molar or mass, and unit-level constraints are written accordingly. The first constraint pertains to the mass transfer term for the aqueous phase. The mass transfer term is equal to $mass\_transfer\_term_{aq} = (D_{i})\frac{mass_{i}~in~aq~phase}{flowrate~of~aq~phase}$. The second constraint relates to the mass transfer term in the liquid phase, which is the negative of the mass transfer term in the aqueous phase: $mass\_transfer\_term_{liq} = - mass\_transfer\_term_{aq} $

This marks the completion of the build function, and the unit model is now equipped with the necessary process constraints. The subsequent steps involve writing the initialization routine.

In [9]:
def build(self):
    """
    Begin building model (pre-DAE transformation).
    Args:
        None
    Returns:
        None
    """
    # Call UnitModel.build to setup dynamics
    super(LiqExtractionData, self).build()

    self.scaling_factor = Suffix(direction=Suffix.EXPORT)

    # Check phase lists match assumptions
    if self.config.aqueous_property_package.phase_list != ["Aq"]:
        raise ConfigurationError(
            f"{self.name} Liquid-Liquid Extractor model requires that the aquoues "
            f"phase property package have a single phase named 'Aq'"
        )
    if self.config.liquid_property_package.phase_list != ["Liq"]:
        raise ConfigurationError(
            f"{self.name} Liquid-Liquid Extractor model requires that the liquid "
            f"phase property package have a single phase named 'Liq'"
        )

    # Check for at least one common component in component lists
    if not any(
        j in self.config.aqueous_property_package.component_list
        for j in self.config.liquid_property_package.component_list
    ):
        raise ConfigurationError(
            f"{self.name} Liquid-Liquid Extractor model requires that the liquid "
            f"and aqueous phase property packages have at least one "
            f"common component."
        )

    self.liquid_phase = ControlVolume0DBlock(
        dynamic=self.config.dynamic,
        has_holdup=self.config.has_holdup,
        property_package=self.config.liquid_property_package,
        property_package_args=self.config.liquid_property_package_args,
    )

    self.liquid_phase.add_state_blocks(
        has_phase_equilibrium=self.config.has_phase_equilibrium
    )

    # Separate liquid and aqueous phases means that phase equilibrium will
    # be handled at the unit model level, thus has_phase_equilibrium is
    # False, but has_mass_transfer is True.

    self.liquid_phase.add_material_balances(
        balance_type=self.config.material_balance_type,
        has_rate_reactions=False,
        has_equilibrium_reactions=self.config.has_equilibrium_reactions,
        has_phase_equilibrium=self.config.has_phase_equilibrium,
        has_mass_transfer=True,
    )

    self.liquid_phase.add_energy_balances(
        balance_type=self.config.energy_balance_type,
        has_heat_transfer=False,
        has_enthalpy_transfer=False,
    )

    self.liquid_phase.add_momentum_balances(
        balance_type=self.config.momentum_balance_type,
        has_pressure_change=self.config.has_pressure_change,
    )

    # ---------------------------------------------------------------------
    self.aqueous_phase = ControlVolume0DBlock(
        dynamic=self.config.dynamic,
        has_holdup=self.config.has_holdup,
        property_package=self.config.aqueous_property_package,
        property_package_args=self.config.aqueous_property_package_args,
    )

    self.aqueous_phase.add_state_blocks(
        has_phase_equilibrium=self.config.has_phase_equilibrium
    )

    # Separate liquid and aqueous phases means that phase equilibrium will
    # be handled at the unit model level, thus has_phase_equilibrium is
    # False, but has_mass_transfer is True.

    self.aqueous_phase.add_material_balances(
        balance_type=self.config.material_balance_type,
        has_rate_reactions=False,
        has_phase_equilibrium=self.config.has_phase_equilibrium,
        has_mass_transfer=True,
    )

    self.aqueous_phase.add_energy_balances(
        balance_type=self.config.energy_balance_type,
        has_heat_transfer=False,
        has_enthalpy_transfer=False,
    )

    self.aqueous_phase.add_momentum_balances(
        balance_type=self.config.momentum_balance_type,
        has_pressure_change=self.config.has_pressure_change,
    )

    # ---------------------------------------------------------------------
    # Check flow basis is compatable
    t_init = self.flowsheet().time.first()
    if (
        self.aqueous_phase.properties_out[t_init].get_material_flow_basis()
        != self.liquid_phase.properties_out[t_init].get_material_flow_basis()
    ):
        raise ConfigurationError(
            f"{self.name} aqueous and liquid property packages must use the "
            f"same material flow basis."
        )

    # Add Ports
    self.add_inlet_port(name="liquid_inlet", block=self.liquid_phase, doc="Liquid feed")
    self.add_inlet_port(
        name="aqueous_inlet", block=self.aqueous_phase, doc="Aqueous feed"
    )
    self.add_outlet_port(
        name="liquid_outlet", block=self.liquid_phase, doc="Liquid Outlet"
    )
    self.add_outlet_port(
        name="aqueous_outlet",
        block=self.aqueous_phase,
        doc="Aqueous Outlet",
    )

    # ---------------------------------------------------------------------
    # Add unit level constraints
    # First, need the union and intersection of component lists
    all_comps = (
        self.aqueous_phase.properties_out.component_list
        | self.liquid_phase.properties_out.component_list
    )
    common_comps = (
        self.aqueous_phase.properties_out.component_list
        & self.liquid_phase.properties_out.component_list
    )

    # Get units for unit conversion
    aunits = self.config.aqueous_property_package.get_metadata().get_derived_units
    lunits = self.config.liquid_property_package.get_metadata().get_derived_units
    flow_basis = self.aqueous_phase.properties_out[t_init].get_material_flow_basis()

    if flow_basis == MaterialFlowBasis.mass:
        fb = "flow_mass"
    elif flow_basis == MaterialFlowBasis.molar:
        fb = "flow_mole"
    else:
        raise ConfigurationError(
            f"{self.name} Liquid-Liquid Extractor only supports mass "
            f"basis for MaterialFlowBasis."
        )

    # Material balances
    def rule_material_liq_balance(self, t, j):
        if j in common_comps:
            return self.aqueous_phase.mass_transfer_term[
                t, "Aq", j
            ] == -self.liquid_phase.config.property_package.diffusion_factor[j] * (
                self.aqueous_phase.properties_in[t].get_mass_comp(j)
                / self.aqueous_phase.properties_in[t].get_flow_rate()
            )
        elif j in self.liquid_phase.properties_out.component_list:
            # No mass transfer term
            # Set Liquid flowrate to an arbitary small value
            return self.liquid_phase.mass_transfer_term[t, "Liq", j] == 0 * lunits(fb)
        elif j in self.aqueous_phase.properties_out.component_list:
            # No mass transfer term
            # Set aqueous flowrate to an arbitary small value
            return self.aqueous_phase.mass_transfer_term[t, "Aq", j] == 0 * aunits(fb)

    self.material_aq_balance = Constraint(
        self.flowsheet().time,
        self.aqueous_phase.properties_out.component_list,
        rule=rule_material_liq_balance,
        doc="Unit level material balances for aq",
    )

    def rule_material_aq_balance(self, t, j):
        print(t)
        if j in common_comps:
            return (
                self.liquid_phase.mass_transfer_term[t, "Liq", j]
                == -self.aqueous_phase.mass_transfer_term[t, "Aq", j]
            )
        else:
            # No mass transfer term
            # Set Liquid flowrate to an arbitary small value
            return self.liquid_phase.mass_transfer_term[t, "Liq", j] == 0 * aunits(fb)

    self.material_liq_balance = Constraint(
        self.flowsheet().time,
        self.liquid_phase.properties_out.component_list,
        rule=rule_material_aq_balance,
        doc="Unit level material balances for Liq",
    )

### Initialization Routine

After writing the unit model it is crucial to develop the initialization routine, as non-linear models may encounter local minima or infeasibility if not initialized properly. Thus, we introduce the function `initialize_build`, serving as the initialization routine for this unit model.

The initialize function accepts `liquid_state_args` and `aqueous_state_args` as inputs, along with the output level for the logger, solver, and solver arguments. The initialization routine unfolds in four steps:

1. Initialize the Liquid Phase: This involves initializing the state variables and constraints associated with the liquid phase.

2. Initialize the Aqueous Phase: Similarly, the state variables and constraints for the aqueous phase are initialized.

3. Solve the Entire Model: The entire model is solved. If the first attempt does not yield an optimal solution, a second attempt is made, and the results are logged.

4. Release the Inlet State Variables: The inlet state variables are released.

After step 4 releases the inlet state variables, and a final check is performed to verify if the results are optimal and an error is raised if the results are not optimal. This four-step process in the initialization routine aims to enhance the likelihood of obtaining a robust and feasible solution for the unit model. This marks the conclusion of creating a custom unit model, for a more detailed explanation on creating a unit model refer [this resource](../../unit_models/custom_unit_models/custom_compressor_doc.md). The next section will deal with the testing of the property package and unit model. 

In [10]:
def initialize_build(
    self,
    liquid_state_args=None,
    aqueous_state_args=None,
    outlvl=idaeslog.NOTSET,
    solver=None,
    optarg=None,
):
    """
    Initialization routine for Liquid Liquid Extractor unit model.

    Keyword Arguments:
        liquid_state_args : a dict of arguments to be passed to the
            liquid property packages to provide an initial state for
            initialization (see documentation of the specific property
            package) (default = none).
        aqueous_state_args : a dict of arguments to be passed to the
            aqueous property package to provide an initial state for
            initialization (see documentation of the specific property
            package) (default = none).
        outlvl : sets output level of initialization routine
        optarg : solver options dictionary object (default=None, use
                 default solver options)
        solver : str indicating which solver to use during
                 initialization (default = None, use default IDAES solver)

    Returns:
        None
    """
    if optarg is None:
        optarg = {}

    # Check DOF
    if degrees_of_freedom(self) != 0:
        raise InitializationError(
            f"{self.name} degrees of freedom were not 0 at the beginning "
            f"of initialization. DoF = {degrees_of_freedom(self)}"
        )

    # Set solver options
    init_log = idaeslog.getInitLogger(self.name, outlvl, tag="unit")
    solve_log = idaeslog.getSolveLogger(self.name, outlvl, tag="unit")

    solverobj = get_solver(solver, optarg)

    # ---------------------------------------------------------------------
    # Initialize liquid phase control volume block
    flags = self.liquid_phase.initialize(
        outlvl=outlvl,
        optarg=optarg,
        solver=solver,
        state_args=liquid_state_args,
        hold_state=True,
    )

    init_log.info_high("Initialization Step 1 Complete.")
    # ---------------------------------------------------------------------
    # Initialize aqueous phase state block
    if aqueous_state_args is None:
        t_init = self.flowsheet().time.first()
        aqueous_state_args = {}
        aq_state_vars = self.aqueous_phase[t_init].define_state_vars()

        liq_state = self.liquid_phase.properties_out[t_init]

        # Check for unindexed state variables
        for sv in aq_state_vars:
            if "flow" in sv:
                aqueous_state_args[sv] = value(getattr(liq_state, sv))
            elif "conc" in sv:
                # Flow is indexed by component
                aqueous_state_args[sv] = {}
                for j in aq_state_vars[sv]:
                    if j in liq_state.component_list:
                        aqueous_state_args[sv][j] = 1e3 * value(
                            getattr(liq_state, sv)[j]
                        )
                    else:
                        aqueous_state_args[sv][j] = 0.5

            elif "pressure" in sv:
                aqueous_state_args[sv] = 1 * value(getattr(liq_state, sv))

            else:
                aqueous_state_args[sv] = value(getattr(liq_state, sv))

    self.aqueous_phase.initialize(
        outlvl=outlvl,
        optarg=optarg,
        solver=solver,
        state_args=aqueous_state_args,
        hold_state=False,
    )

    init_log.info_high("Initialization Step 2 Complete.")

    # ---------------------------------------------------------------------
    # # Solve unit model
    with idaeslog.solver_log(solve_log, idaeslog.DEBUG) as slc:
        results = solverobj.solve(self, tee=slc.tee)
        if not check_optimal_termination(results):
            init_log.warning(
                f"Trouble solving unit model {self.name}, trying one more time"
            )
            results = solverobj.solve(self, tee=slc.tee)
    init_log.info_high("Initialization Step 3 {}.".format(idaeslog.condition(results)))

    # ---------------------------------------------------------------------
    # Release Inlet state
    self.liquid_phase.release_state(flags, outlvl)
    self.aqueous_phase.release_state(flags, outlvl)
    if not check_optimal_termination(results):
        raise InitializationError(
            f"{self.name} failed to initialize successfully. Please check "
            f"the output logs for more information."
        )

    init_log.info("Initialization Complete: {}".format(idaeslog.condition(results)))

# 4. Testing

There are typically 3 types of tests:

1. `Unit tests`: Test runs quickly (under 2 seconds) and has no network/system dependencies. Uses only libraries installed by default with the software
2. `Component test`: Test may run more slowly (under 10 seconds, or so), e.g. it may run a solver or create a bunch of files. Like unit tests, it still shouldn't depend on special libraries or dependencies.
3. `Integration test`: Test may take a long time to run, and may have complex dependencies.

The expectation is that unit tests should be run by developers rather frequently, component tests should be run by the continuous integration system before running code, and integration tests are run across the codebase regularly, but infrequently (e.g. daily).


As a developer, testing is a crucial aspect of ensuring the reliability and correctness of the unit model. The testing process involves both Unit tests and Component tests, and pytest is used as the testing framework. A typical test is marked with @pytest.mark.level, where the level indicates the depth or specificity of the testing. This is written in a file usually named as test_*.py or *_test.py. The test files have functions written in them with the appropriate lavel of test being conducted.  

For more detailed information on testing methodologies and procedures, developers are encouraged to refer to [this resource](https://idaes-pse.readthedocs.io/en/stable/reference_guides/developer/testing.html). The resource provides comprehensive guidance on the testing process and ensures that the unit model meets the required standards and functionality.

## 4.1 Property package
### Unit Tests

When writing tests for the Aqueous property phase package, it's essential to focus on key aspects to ensure the correctness and robustness of the implementation. Here are the areas to cover in the unit tests:

1. Number of Config Dictionaries: Verify that the property phase package has the expected number of configuration dictionaries.

2. State Block Class Name: Confirm that the correct state block class is associated with the Aqueous property phase package.

3. Number of Phases: Check that the Aqueous property phase package defines the expected number of phases.

4. Components in the Phase and Physical Parameter Values: Test that the components present in the Aqueous phase match the anticipated list. Additionally, validate that the physical parameter values (such as density, viscosity, etc.) are correctly defined.


In [11]:
import pytest
from pyomo.environ import ConcreteModel, Param, value, Var
from pyomo.util.check_units import assert_units_consistent
from idaes.core import MaterialBalanceType, EnergyBalanceType

from Liq_property import LiqPhase
from Aq_property import AqPhase
from liquid_extraction import LiqExtraction
from idaes.core.solvers import get_solver

solver = get_solver()


class TestParamBlock(object):
    @pytest.fixture(scope="class")
    def model(self):
        model = ConcreteModel()
        model.params = LiqPhase()
        return model

    @pytest.mark.unit
    def test_config(self, model):
        assert len(model.params.config) == 1

    @pytest.mark.unit
    def test_build(self, model):
        assert model.params.state_block_class is AqPhaseStateBlock

        assert len(model.params.phase_list) == 1
        for i in model.params.phase_list:
            assert i == "Aq"

        assert len(model.params.component_list) == 4
        for i in model.params.component_list:
            assert i in ["H2O", "NaCl", "KNO3", "CaSO4"]

        assert isinstance(model.params.cp_mol, Param)
        assert value(model.params.cp_mol) == 4182

        assert isinstance(model.params.dens_mol, Param)
        assert value(model.params.dens_mol) == 997

        assert isinstance(model.params.temperature_ref, Param)
        assert value(model.params.temperature_ref) == 298.15

The next set of unit tests focuses on testing the build function in the state block. Here are the key aspects to cover in these tests:

1. Existence and Initialized Values of State Variables: Verify that the state variables are correctly defined and initialized within the state block. This ensures that the state block is properly constructed and ready for initialization.

2. Initialization Function Test: Check that state variables are not fixed before initialization and are released after initialization. This test ensures that the initialization process occurs as expected and that the state variables are appropriately managed throughout.

These unit tests provide comprehensive coverage for validating the functionality and behavior of the state block in the Aqueous property phase package. Similar tests can be written for the liquid property package to ensure consistency and reliability across both packages.

In [12]:
class TestStateBlock(object):
    @pytest.fixture(scope="class")
    def model(self):
        model = ConcreteModel()
        model.params = AqPhase()

        model.props = model.params.build_state_block([1])

        return model

    @pytest.mark.unit
    def test_build(self, model):
        assert isinstance(model.props[1].flow_vol, Var)
        assert value(model.props[1].flow_vol) == 1

        assert isinstance(model.props[1].temperature, Var)
        assert value(model.props[1].temperature) == 300

        assert isinstance(model.props[1].conc_mass_comp, Var)
        assert len(model.props[1].conc_mass_comp) == 3

        for i in model.props[1].conc_mass_comp:
            print(value(model.props[1].conc_mass_comp[i]))
            assert value(model.props[1].conc_mass_comp[i]) == 1

    @pytest.mark.unit
    def test_initialize(self, model):
        assert not model.props[1].flow_vol.fixed
        assert not model.props[1].temperature.fixed
        assert not model.props[1].pressure.fixed
        for i in model.props[1].conc_mass_comp:
            assert not model.props[1].conc_mass_comp[i].fixed

        model.props.initialize(hold_state=False, outlvl=1)

        assert not model.props[1].flow_vol.fixed
        assert not model.props[1].temperature.fixed
        assert not model.props[1].pressure.fixed
        for i in model.props[1].conc_mass_comp:
            assert not model.props[1].conc_mass_comp[i].fixed

### Component Tests
In the component test, we aim to ensure unit consistency across the entire property package. Unlike unit tests that focus on individual functions, component tests assess the coherence and consistency of the entire package. Here's what the component test will entail:

Unit Consistency Check: Verify that all units used within the property package are consistent throughout. This involves checking that all parameters, variables, and equations within the package adhere to the same unit system, ensuring compatibility.

By conducting a comprehensive component test, we can ensure that the property package functions as a cohesive unit, maintaining consistency and reliability across its entirety. This concludes our tests on the property package. Next we shall test the unit model. 

In [13]:
@pytest.mark.component
def check_units(model):
    model = ConcreteModel()
    model.params = AqPhase()
    assert_units_consistent(model)

# 4.2 Unit Model
### Unit tests
Unit tests for the unit model encompass verifying the configuration arguments and the build function, similar to the approach taken for the property package. When testing the config arguments, we ensure that the correct number of arguments is provided and then match each argument with the expected one. This ensures that the unit model is properly configured and ready to operate as intended.

In [14]:
import pytest

import idaes.core
import idaes.models.unit_models
from idaes.core.solvers import get_solver
import idaes.logger as idaeslog


from pyomo.environ import value, check_optimal_termination, units
from pyomo.util.check_units import assert_units_consistent
from idaes.core.util.model_statistics import (
    number_variables,
    number_total_constraints,
)
from idaes.core.solvers import get_solver
from idaes.core.initialization import (
    SingleControlVolumeUnitInitializer,
)

solver = get_solver()


@pytest.mark.unit
def test_config():
    m = ConcreteModel()
    m.fs = idaes.core.FlowsheetBlock(dynamic=False)
    m.fs.liq_properties = LiqPhase()
    m.fs.aq_properties = AqPhase()

    m.fs.unit = LiqExtraction(
        dynamic=False,
        has_pressure_change=False,
        liquid_property_package=m.fs.liq_properties,
        aqueous_property_package=m.fs.aq_properties,
    )

    # Check unit config arguments
    assert len(m.fs.unit.config) == 16

    # Check for config arguments
    assert m.fs.unit.config.material_balance_type == MaterialBalanceType.useDefault
    assert m.fs.unit.config.energy_balance_type == EnergyBalanceType.useDefault
    assert m.fs.unit.config.momentum_balance_type == MomentumBalanceType.pressureTotal
    assert not m.fs.unit.config.has_heat_transfer
    assert not m.fs.unit.config.has_pressure_change
    assert not m.fs.unit.config.has_equilibrium_reactions
    assert not m.fs.unit.config.has_phase_equilibrium
    assert not m.fs.unit.config.has_heat_of_reaction
    assert m.fs.unit.config.liquid_property_package is m.fs.liq_properties
    assert m.fs.unit.config.aqueous_property_package is m.fs.aq_properties

    # Check for unit initializer
    assert m.fs.unit.default_initializer is SingleControlVolumeUnitInitializer

In testing the build function, we verify whether the number of variables aligns with the intended values and also check for the existence of desired constraints within the unit model. This ensures that the unit model is constructed accurately and includes all the necessary variables and constraints required for its proper functioning.

In [15]:
class TestBuild(object):
    @pytest.fixture(scope="class")
    def model(self):
        m = ConcreteModel()
        m.fs = idaes.core.FlowsheetBlock(dynamic=False)
        m.fs.liq_properties = LiqPhase()
        m.fs.aq_properties = AqPhase()

        m.fs.unit = LiqExtraction(
            dynamic=False,
            has_pressure_change=False,
            liquid_property_package=m.fs.liq_properties,
            aqueous_property_package=m.fs.aq_properties,
        )

        m.fs.unit.liquid_inlet.flow_vol.fix(80 * units.ml / units.min)
        m.fs.unit.liquid_inlet.temperature.fix(300 * units.K)
        m.fs.unit.liquid_inlet.pressure.fix(1 * units.atm)
        m.fs.unit.liquid_inlet.conc_mass_comp[0, "NaCl"].fix(0 * units.g / units.kg)
        m.fs.unit.liquid_inlet.conc_mass_comp[0, "KNO3"].fix(0 * units.g / units.kg)
        m.fs.unit.liquid_inlet.conc_mass_comp[0, "CaSO4"].fix(0 * units.g / units.kg)

        m.fs.unit.aqueous_inlet.flow_vol.fix(10 * units.ml / units.min)
        m.fs.unit.aqueous_inlet.temperature.fix(300 * units.K)
        m.fs.unit.aqueous_inlet.pressure.fix(1 * units.atm)
        m.fs.unit.aqueous_inlet.conc_mass_comp[0, "NaCl"].fix(0.15 * units.g / units.kg)
        m.fs.unit.aqueous_inlet.conc_mass_comp[0, "KNO3"].fix(0.2 * units.g / units.kg)
        m.fs.unit.aqueous_inlet.conc_mass_comp[0, "CaSO4"].fix(0.1 * units.g / units.kg)

        return m

    @pytest.mark.build
    @pytest.mark.unit
    def test_build(self, model):

        assert hasattr(model.fs.unit, "aqueous_inlet")
        assert len(model.fs.unit.aqueous_inlet.vars) == 4
        assert hasattr(model.fs.unit.aqueous_inlet, "flow_vol")
        assert hasattr(model.fs.unit.aqueous_inlet, "conc_mass_comp")
        assert hasattr(model.fs.unit.aqueous_inlet, "temperature")
        assert hasattr(model.fs.unit.aqueous_inlet, "pressure")

        assert hasattr(model.fs.unit, "liquid_inlet")
        assert len(model.fs.unit.liquid_inlet.vars) == 4
        assert hasattr(model.fs.unit.liquid_inlet, "flow_vol")
        assert hasattr(model.fs.unit.liquid_inlet, "conc_mass_comp")
        assert hasattr(model.fs.unit.liquid_inlet, "temperature")
        assert hasattr(model.fs.unit.liquid_inlet, "pressure")

        assert hasattr(model.fs.unit, "aqueous_outlet")
        assert len(model.fs.unit.aqueous_outlet.vars) == 4
        assert hasattr(model.fs.unit.aqueous_outlet, "flow_vol")
        assert hasattr(model.fs.unit.aqueous_outlet, "conc_mass_comp")
        assert hasattr(model.fs.unit.aqueous_outlet, "temperature")
        assert hasattr(model.fs.unit.aqueous_outlet, "pressure")

        assert hasattr(model.fs.unit, "liquid_outlet")
        assert len(model.fs.unit.liquid_outlet.vars) == 4
        assert hasattr(model.fs.unit.liquid_outlet, "flow_vol")
        assert hasattr(model.fs.unit.liquid_outlet, "conc_mass_comp")
        assert hasattr(model.fs.unit.liquid_outlet, "temperature")
        assert hasattr(model.fs.unit.liquid_outlet, "pressure")

        assert hasattr(model.fs.unit, "material_aq_balance")
        assert hasattr(model.fs.unit, "material_liq_balance")

        assert number_variables(model) == 34
        assert number_total_constraints(model) == 19

### Component tests

During the component tests, we evaluate the performance of the unit model when integrated with the property package. This evaluation process typically involves several steps:

1. Unit Consistency Check: Verify that the unit model maintains consistency in its units throughout the model. This ensures that all variables and constraints within the model adhere to the same unit system, guaranteeing compatibility.

2. Termination Condition Verification: This involves checking whether the model terminates optimally with the given inlet conditions.

3. Variable Value Assessment: Check the values of outlet variables against the expected values. To account for the numerical tolerance of the solvers, the values are compared using the approx function with a relative tolerance.

4. Input Variable Stability Test: Verify that input variables, which should remain fixed during model operation, are not inadvertently unfixed or altered. 

By performing these checks, we conclude the testing for the unit model. 

In [17]:
class TestFlowsheet:
    @pytest.fixture
    def model(self):
        m = ConcreteModel()
        m.fs = idaes.core.FlowsheetBlock(dynamic=False)
        m.fs.liq_properties = LiqPhase()
        m.fs.aq_properties = AqPhase()

        m.fs.unit = LiqExtraction(
            dynamic=False,
            has_pressure_change=False,
            liquid_property_package=m.fs.liq_properties,
            aqueous_property_package=m.fs.aq_properties,
        )
        m.fs.unit.liquid_inlet.flow_vol.fix(80 * units.ml / units.min)
        m.fs.unit.liquid_inlet.temperature.fix(300 * units.K)
        m.fs.unit.liquid_inlet.pressure.fix(1 * units.atm)
        m.fs.unit.liquid_inlet.conc_mass_comp[0, "NaCl"].fix(0 * units.g / units.kg)
        m.fs.unit.liquid_inlet.conc_mass_comp[0, "KNO3"].fix(0 * units.g / units.kg)
        m.fs.unit.liquid_inlet.conc_mass_comp[0, "CaSO4"].fix(0 * units.g / units.kg)

        m.fs.unit.aqueous_inlet.flow_vol.fix(10 * units.ml / units.min)
        m.fs.unit.aqueous_inlet.temperature.fix(300 * units.K)
        m.fs.unit.aqueous_inlet.pressure.fix(1 * units.atm)
        m.fs.unit.aqueous_inlet.conc_mass_comp[0, "NaCl"].fix(0.15 * units.g / units.kg)
        m.fs.unit.aqueous_inlet.conc_mass_comp[0, "KNO3"].fix(0.2 * units.g / units.kg)
        m.fs.unit.aqueous_inlet.conc_mass_comp[0, "CaSO4"].fix(0.1 * units.g / units.kg)

        return m

    @pytest.mark.component
    def test_unit_model(self, model):
        assert_units_consistent(model)
        solver = get_solver()
        results = solver.solve(model, tee=False)

        # Check for optimal termination
        assert check_optimal_termination(results)

        # Checking for outlet flows
        assert value(model.fs.unit.liquid_outlet.flow_vol[0]) == pytest.approx(
            80.0, rel=1e-5
        )
        assert value(model.fs.unit.aqueous_outlet.flow_vol[0]) == pytest.approx(
            10.0, rel=1e-5
        )

        # Checking for outlet mass_comp
        assert value(
            model.fs.unit.liquid_outlet.conc_mass_comp[0, "CaSO4"]
        ) == pytest.approx(0.000187499, rel=1e-5)
        assert value(
            model.fs.unit.liquid_outlet.conc_mass_comp[0, "KNO3"]
        ) == pytest.approx(0.000749999, rel=1e-5)
        assert value(
            model.fs.unit.liquid_outlet.conc_mass_comp[0, "NaCl"]
        ) == pytest.approx(0.000403124, rel=1e-5)
        assert value(
            model.fs.unit.aqueous_outlet.conc_mass_comp[0, "CaSO4"]
        ) == pytest.approx(0.0985, rel=1e-5)
        assert value(
            model.fs.unit.aqueous_outlet.conc_mass_comp[0, "KNO3"]
        ) == pytest.approx(0.194, rel=1e-5)
        assert value(
            model.fs.unit.aqueous_outlet.conc_mass_comp[0, "NaCl"]
        ) == pytest.approx(0.146775, rel=1e-5)

        # Checking for outlet temperature
        assert value(model.fs.unit.liquid_outlet.temperature[0]) == pytest.approx(
            300, rel=1e-5
        )
        assert value(model.fs.unit.aqueous_outlet.temperature[0]) == pytest.approx(
            300, rel=1e-5
        )

        # Checking for outlet pressure
        assert value(model.fs.unit.liquid_outlet.pressure[0]) == pytest.approx(
            1, rel=1e-5
        )
        assert value(model.fs.unit.aqueous_outlet.pressure[0]) == pytest.approx(
            1, rel=1e-5
        )

        # Fixed state variables
        assert model.fs.unit.liquid_inlet.flow_vol[0].fixed
        assert model.fs.unit.liquid_inlet.conc_mass_comp[0, "NaCl"].fixed
        assert model.fs.unit.liquid_inlet.conc_mass_comp[0, "KNO3"].fixed
        assert model.fs.unit.liquid_inlet.conc_mass_comp[0, "CaSO4"].fixed
        assert model.fs.unit.liquid_inlet.temperature[0].fixed
        assert model.fs.unit.liquid_inlet.pressure[0].fixed

        assert model.fs.unit.aqueous_inlet.flow_vol[0].fixed
        assert model.fs.unit.aqueous_inlet.conc_mass_comp[0, "NaCl"].fixed
        assert model.fs.unit.aqueous_inlet.conc_mass_comp[0, "KNO3"].fixed
        assert model.fs.unit.aqueous_inlet.conc_mass_comp[0, "CaSO4"].fixed
        assert model.fs.unit.aqueous_inlet.temperature[0].fixed
        assert model.fs.unit.aqueous_inlet.pressure[0].fixed